# Window Functions in SQL

![](https://i.imgur.com/63oIWPU.png)

The following topics are covered in this tutorial:

- Aggregation vs Window Functions
- Database & system setup
- Problem solving approach
- Window function basics
- Aggregation, Ranking & Value functions
- Advanced windowing 

Window functions in SQL are used to partition rows of a table into multiple "windows" to perform separate computation for each window that returns a new value for each row of the table.

Windowing is a powerful and flexible feature of SQL which allows to perform many different kinds of operations & analysis which are not possible otherwise. 


## Aggregation vs Window Functions


![](https://i.imgur.com/Z5RNWdC.png)

Unlike grouping and aggregation functions, window functions do not combine input rows in to a single output row. Instead, each input row is retained in the output and a new value is added in each row which is computed using data from all the rows in the window. 

## Database & System Setup


### Database Setup


In this tutorial, we'll use the [Classic Models database](https://www.mysqltutorial.org/mysql-sample-database.aspx) from [the previous tutorial](https://jovian.ai/aakashns/relational-databases-and-sql). To set up the database locally with sample data:

1. Download this [SQL file](https://raw.githubusercontent.com/harsha547/ClassicModels-Database-Queries/master/database.sql)
2. In MySQL Workbench, click "File" > "Open SQL Script" to open the script;
3. Execute the script to create and populate the database. 

Once executed, you should be able to view and browse tables in the "Schema" section of the sidebar. If you face an error, make sure you have MySQL server running. 

> Classic Models Inc. is a distributor of small scale models of cars, motorcycles, planes, ships trains etc. Products manufactured by Classic Models are sold in toy & gift stores around the world. Here's a small sample of their products ([source](https://tinytown.in/)):
>
> ![](https://i.imgur.com/9F9WbbA.png)
> 
> Classic Models has offices around the world with dozens of employees. The customers of Classic Models are typically toy/gift stores. Each customer has a designated sales representative (an employee of Classic Models) they interact with. Customers typically place orders requesting several products in different quantities and pay for multiple orders at once via cheques.


Here's the Entity Relationship Diagram (ERD) for the database:

![](https://i.imgur.com/H6q1dAb.png)



### Setting up MySQL Server Locally

We'll use the MySQL server for this tutorial. Make sure to install the following on your computer:

- MySQL server: https://dev.mysql.com/downloads/mysql/
- MySQL workbench: https://dev.mysql.com/downloads/workbench/

You'll be asked to set a root password while installing MySQL server.

To interact with the MySQL server via the terminal use:

```
$ /usr/local/mysql/bin/mysql -u root -p
```

Depending on your operating system the path `/usr/local/mysql/bin/mysql` may be different. If you're unable to connect, make sure that the server is running and you're using the correct password. 

Alternatively, the [MySQL Workbench](https://www.mysql.com/products/workbench/) can be used to interact with a MySQL server (local or remote) via a GUI.

<img src="https://i.imgur.com/LaHS8x0.png" width="640" style="border-radius:4px">


## Problem Solving Approach

In this lesson, we will be working with some problems that you will come across in coding challenges, interviews or at work when working with databases. 

How do we approach these problems?
- <b>Understand inputs & outputs</b>: Look at the problem description and identify what goes in (inputs) and what comes out (outputs) i.e. which tables are required and what columns are wanted in the output
- <b>Articulate solution verbally</b>: When working with databases or in an interview, before writing the SQL code, one should talk through the problem and explain the approach about how to build the solution 
- <b>Construct the query step-by-step</b>: The SQL query should always by constructed step-by-step, get the intermediate results and continue improving the query 

## Window Function Basics

Let us start with the basics of window function!

### Window Function Syntax

A window function is usually a part of `SELECT` statement and is used to add a new column into the result of the `SELECT` statement


```

<function type>  OVER (

PARTITION BY clause

ORDER BY clause

ROWS or RANGE clause

) AS column_name


```

- Window function clause typically starts with type of the function or computation to be applied. For example, each window can compute `SUM`, `COUNT`, `AVG`, `RANK`, `ROW_NUMBER` etc.
- Function or computation is followed by the `OVER` keyword, an indication to SQL to create windows. 
- Inside`()`, an optional `PARTITION BY` clause can be provided to decide how the rows of the table are partitioned into windows.
- An optional `ORDER BY` clause can also be provided to determine the order of the rows in the each window.
- A `ROWS` or `RANGE` clause can be added to limit the size of the window frame while performing computation for each row.
- Finally, `AS` keyword can be used to give a name to the resulting column or an alias in the output.


### Window Function Execution

`OVER`, `PARTITION BY` and `ORDER BY` are the three main clauses that form the backbone of a window function.


#### <b> Overview of Window Function Execution 📙</b>


![](https://i.imgur.com/WCZO9Sc.png)


#### ```OVER``` Clause
   - It is the core of any window function. 
   - It is required in every window function.
   - It is preceded by function type like `ROW_NUMBER`, `MAX`, `MIN` etc.
   - It is followed by selection clauses like `PARTITION BY`, `ORDER BY`, `ROWS BETWEEN` etc.

> <b> QUESTION </b>: Show the list of all the payments made by the customers of Classic Models, and assign a unique payment number to each payment.


```
SELECT *,
  ROW_NUMBER()
  OVER ()
  AS paymentNumber
FROM payments;

```

#### ```PARTITION BY``` Clause 
   - It is optional but commonly used clause in window functions.
   - It is used to group rows in windows.
   - It is followed by column name.

> <b> QUESTION </b>: Express each payment made by a customer as a fraction of the total payment made by the customer. Show the checkNumber, customerNumber, paymentDate, amount & fractionOfTotal.

```

SELECT *, amount/ (
   SUM(amount) OVER
   (PARTITION BY customerNumber)
 ) AS fractionOfTotal
FROM payments;

```

#### ```ORDER BY```  Clause
  - This clause determines the order of rows within a window.
  - It is also followed by column name.
  - It is used to limit frame to current row.

> <b> QUESTION </b>: Show the running total payments received by Classic Models, starting from the oldest to the newest. Show the checkNumber, customerNumber, paymentDate, amount & runningTotal.


```

SELECT *,
  SUM(amount) OVER
  (ORDER BY paymentDate)
  AS runningTotal
FROM payments;

```

![](https://i.imgur.com/PqMJGlJ.png)

## Aggregation, Ranking & Value Functions

Once it is decided on how to create the windows, it's time to decide what type of function needs to be applied to each window. The different types of functions that can be applied to each window are aggregation, ranking & value functions. 


### Aggregation Functions

The functions are the usual aggregation functions that we have in SQL.

#### ```SUM, AVG```

> <b> QUESTION </b>: Show the running total and running average of payments by each customer, starting from the oldest to the newest. Also output the customerNumber, and paymentDate.

```

SELECT *, SUM(amount) OVER (
  PARTITION BY customerNumber
  ORDER BY paymentDate)
AS customerRunningTotal,
AVG(amount) OVER (
  PARTITION BY customerNumber
  ORDER BY paymentDate)
AS customerRunningAverage FROM payments;

```

#### ```COUNT```

> <b> QUESTION </b>: Determine the team size for each employee i.e. how many people report to the same manager as the employee. Output employeeNumber, lastName, firstName, reportsTo, and teamSize.

```

SELECT employeeNumber, lastName,
  firstName, reportsTo,
  COUNT(*) OVER
  (PARTITION BY reportsTo)
  AS teamSize
FROM employees;

```

#### `MAX` & `MIN`

> <b> QUESTION </b>: For each order, show the order number, total order amount and the difference between the order amount and the maximum/minimum order amount for the customer.

```

SELECT orderNumber, customerNumber, orderValue,
  ROUND(MAX(orderValue) OVER (PARTITION BY customerNumber) 
    - orderValue, 2) AS belowMax,
  ROUND(orderValue - MIN(orderValue)
    OVER (PARTITION BY customerNumber), 2) AS aboveMin
FROM (SELECT o.orderNumber, o.customerNumber,
    SUM(od.quantityOrdered * od.priceEach) AS orderValue
 FROM orders o JOIN orderdetails od
 ON o.orderNumber = od.orderNumber
 GROUP BY o.orderNumber, o.customerNumber)
AS ordersWithValues;

```

### Ranking Functions

Ranking functions take the rows within a window and give them ranks or numbers.

#### ```ROW_NUMBER```
   - `ROW_NUMBER` function is a SQL ranking function that assigns a sequential rank number to each new record in a partition. When `ROW_NUMBER` function detects two identical values in the same partition, it assigns different rank numbers to both.

> <b> QUESTION </b>: Provide serial numbers for the customers of each sales representative, sorted by dates of each customer's earliest orders.

```
SELECT customerNumber, customerName,
  salesRepEmployeeNumber, earliestOrder,
  ROW_NUMBER() OVER (PARTITION BY salesRepEmployeeNumber 
  ORDER BY earliestOrder) AS serialNumber
FROM (SELECT c.customerNumber, c.customerName,
   c.salesRepEmployeeNumver,
   MIN(o.orderDate) AS earliestOrder
  FROM customers c JOIN order o
  ON c.customerNumber = o.customerNumber
  WHERE c.salesRepEmployeeNumber IS NOT NULL
  GROUP BY c.customerNumber, c.salesRepEmployeeNumber 
) AS co;

```

#### `RANK`, `DENSE_RANK` & `CUME_DIST`

   - `RANK()`: The `RANK()` window function calculates a rank for each row within a partition of a table. If the rows in a partition have the same values, same rank is assigned to them.

   - `DENSE_RANK()`: The `DENSE_RANK()` window function assigns a rank to each row within a partition of a table. Unlike the `RANK()` function, the `DENSE_RANK()` function returns consecutive rank values. If the rows in a partition have the same values, same rank is assigned to them.

   - `CUME_DIST()`: The `CUME_DIST()` window function calculates the cumulative distribution of value within a set of values.

> <b> QUESTION </b>: Rank the offices of Classic Models based on the number of employees (lowest first). Compare the outputs of `ROW_NUMBER`,`RANK`, `DENSE_RANK` and `CUME_DIST`.

```

SELECT *, 
ROW_NUMBER() OVER (ORDER BY employeeCount) AS rowNumber,
RANK () OVER (ORDER BY employeeCount) AS rankNumber,
DENSE_RANK() OVER (ORDER BY employeeCount) AS
denseRankNumber,
CUME_DIST() OVER (ORDER BY employeeCount) AS cumeDist
FROM (SELECT o.officeCode, city, COUNT(*) AS employeeCountt
  FROM offices o JOIN employees e
  ON o.officeCode = e.officeCode
  GROUP BY o.officeCode, city) AS oe;
  
```

#### `NTILE` & `PERCENT_RANK`
   - These functions are used to compute percentiles and quartiles.
   - `NTILE`: It puts rows into buckets in a window.
   - `PERCENT_RANK`: The `PERCENT_RANK` function calculates the relative rank percentile of each row.
   - These functions are typically used with `ORDER BY` clause.
   

> <b> QUESTION </b>: Compute the quartile and percentile for each payment made by a customer. Output checkNumber, paymentDate, amount, customerNumber, quartile, and percentRank.

```

SELECT *, 
  NTILE(4) OVER
  (PARTITION BY customerNumber
  ORDER BY amount) AS quartile,
  PERCENT_RANK() OVER
  (PARTITION BY customerNumber
  ORDER BY amount) AS percentRank
RANK payments;

```

### Value Functions

Value functions pick specific values from within a window and add them into the new column. These are useful when you need to compare the value in a certain row/column with a value in the previous row/column, generally in time series data. 

#### ```LAG```

   - It used to access previous row data 

> <b> QUESTION </b>: For each order, show the number of days elapsed since the last order places by the same customer. Output OrderNumber, orderDate, customerNumber, and daysSinceLastOrder.

```

SELECT orderNumber, orderDate,
customerNumber,
  DATEDIFF(orderDate, 
    LAG(orderDate, 1) OVER
    (PARTITION BY customerNumber
    ORDER BY orderDate)
  ) AS daysSinceLastOrder
FROM orders;

```

#### ```LEAD```
   - It is used to access next row data

> <b> QUESTION </b>: List order of each customer sorted by order amount (highest to lowest) and display the difference in amount between each order and the next highest order by the same customer.

```

SELECT *,
  orderValue - LEAD(orderValue, 1) OVER
  (PARTITION BY customerNumber
  ORDER BY orderValue DESC) AS higherBy
FROM (SELECT o.orderNumber, o.orderDate, o.customerNumber,
    SUM(od.quantityOrdered * od.priceEach) AS orderValue
  FROM orders o JOIN orderdetails od
  ON o.orderNumber = od.orderNumber
  GROUP BY o.orderNumber, o.customerNumber
AS ordersWithValues;

```

#### ```FIRST_VALUE, LAST_VALUE, NTH_VALUE```
   - It is used to access first, last or nth row from the partitioned window

> <b> QUESTION </b>: For each payment made by a customer compute the difference between the payment amount and the first/second/last payment made by the customer.

```
SELECT *,
  ROUND(amount - FIRST_VALUE(amount)
    OVER customersWindow, 2) AS diffFirst,
  ROUND(amount - LAST_VALUE(amount)
    OVER customersWindow, 2) AS diffLast,
  ROUND(amount - NTH_VALUE(amount,2)
    OVER customersWindow, 2) AS diffSecond
FROM payments WINDOW customersWindow AS
  (PARTITION BY customerNumber ORDER BY amount DESC)
  ROWS BETWEEN UNBOUNDED PRECEDING
  AND UNBOUNDED FOLLOWING);

```

## Advanced Windowing 

Some advanced topics in windowing are:

- `ROWS BETWEEN` and `RANGE BETWEEN` clauses that are required to select the frame of rows that get used in a particular computation.
- `WINDOW` Alias clause is required to create an alias when using the same window function in different queries.

### ```ROWS BETWEEN``` Clause

   - It is used to specify the window frame in relation to the current row
   - <b>Syntax</b>: `ROWS BETWEEN lower_bound AND upper_bound`
   - There are five options for bounds:
        - `UNBOUNDED PRECEDING`: All rows before the current row
        - `n PRECEDING`: <i>n</i> rows <b>before</b> the current row
        - `CURRENT ROW`: Just the current row
        - `n FOLLOWING`: <i>n</i> rows <b>after</b> the current row
        - `UNBOUNDED FOLLOWING`: All rows after the current row
          
![](https://i.imgur.com/PqMJGlJ.png)


5 Practical Examples of Using `ROWS BETWEEN` in SQL: https://learnsql.com/blog/sql-window-functions-rows-clause/

### ```RANGE BETWEEN``` Clause

   - `ROWS BETWEEN` allows you to specify the number of rows before and after you want to pick where are `RANGE BETWEEN` works with the actual value.
   
![](https://i.imgur.com/5nFOOoT.png)


5 Practical Examples of Using `RANGE BETWEEN` in SQL: https://learnsql.com/blog/range-clause/

### ```WINDOW``` Alias

   - If you need to use the same partitioning logic multiple times then you can just give the logic a name and use it for the window functions over the same division 

```
SELECT start_terminal,
       duration_seconds,
       NTILE(4) OVER ntile_window AS quartile, 
       NTILE(5) OVER ntile_window AS quintile,
       NTILE(100) OVER ntile_window AS percentile
  FROM tutorial.dc_bikeshare_ql_2012
 WHERE start_time < '2012-02-08'
WINDOW ntile_window AS
         (PARTITION BY start_terminal ORDER BY duration_seconds)
 ORDER BY start_terminal, duration_seconds
 
```

## Practice Problems

### Amazon Interview Question

> <b> QUESTION </b>: Find the percentage of the total spend a customer spent on each order on Amazon. Output the customer's name, order details, and the percentage of thier total spend for each order transaction rounded to the nearest whole number.

![](https://i.imgur.com/2rtvgfZ.png)

```

SELECT c.name, o.order_details, 
  ROUND(o,order_cost * 100 /
    CAST(SUM(o.order_cost) OVER
    (PARTITION BY c.name) AS FLOAT)
  ) AS percentage_of_total
FROM orders o JOIN customers c
  ON c.id = o.customer_id
ORDER BY c.name;

```

### Uber Interview Question

> <b> QUESTION </b>: You’re given a dataset of uber rides with the traveling distance (‘distance_to_travel’) and cost (‘monetary_cost’) for each ride. For each date, find the difference between the distance-per-dollar for that date and the average distance-per-dollar for that year-month. Distance-per-dollar is defined as the distance traveled divided by the cost of the ride.

*The output should include the year-month (YYYY-MM) and the absolute average difference in distance-per-dollar (Absolute value to be rounded to the 2nd decimal). 
You should also count both success and failed request_status as the distance and cost values are populated for all ride requests. Also, assume that all dates are unique in the dataset. Order your results by earliest request date first.*

![](https://i.imgur.com/UoNHsF1.png)

### Airbnb Interview Question

> <b> QUESTION </b>: Estimate the growth of Airbnb each year using the number of hosts registered as the growth metric. 

` rate_of_growth = 100 * (hosts_registered_this_year - hosts_registered_last_year) / hosts_registered_last_year `

*Output the year, number of hosts in the current year, number of hosts in the previous year, and the rate of growth. Round the rate of growth to the nearest percent and order the result in the ascending order based on the year.
Assume that the dataset consists only of unique hosts, meaning there are no duplicate hosts listed.*


![](https://i.imgur.com/PSvxbkG.png)

### Twitter Interview Question

> <b> QUESTION </b>: Find the top three unique salaries for each department. Output the department name and the top 3 unique salaries by each department. Order your results alphabetically by department and then by highest salary to lowest.

![](https://i.imgur.com/rrBRmi2.png)

### Netflix Interview Question

> <b> QUESTION </b>: ABC Corp is a mid-sized insurer in the US and in the recent past their fraudulent claims have increased significantly for their personal auto insurance portfolio. They have developed a ML based predictive model to identify propensity of fraudulent claims. Now, they assign highly experienced claim adjusters for top 5 percentile of claims identified by the model. Your objective is to identify the top 5 percentile of claims from each state. 

*Your output should be policy number, state, claim cost, and fraud score.*

![](https://i.imgur.com/sG64cfJ.png) 

## Summary

The following topics are covered in this tutorial:

- Aggregation vs Window Functions
- Database & system setup
- Problem solving approach
- Window function basics
- Aggregation, Ranking & Value functions
- Advanced windowing 
- Practice problems

## Resources 

Check out the following resources to learn more:

- ClassicModels Database: https://www.mysqltutorial.org/mysql-sample-database.aspx
- LearnSQL.com: https://learnsql.com/
- Mode SQL Tutorial: https://mode.com/sql-tutorial/
- StrataScratch: https://www.stratascratch.com/
- 5 Practical Examples of Using `ROWS BETWEEN` in SQL: https://learnsql.com/blog/sql-window-functions-rows-clause/
- 5 Practical Examples of Using `RANGE BETWEEN` in SQL: https://learnsql.com/blog/range-clause/
- Solutions for the interview questions: https://www.youtube.com/watch?v=XBE09l-UYTE
- Relational databases book: https://db-book.com

## Revision Questions

1. What are window functions in SQL?
2. What are the uses of window functions in SQL?
3. How are window functions different from aggregate functions?
4. How to approach a SQL problem?
5. What is window function syntax?
6. What are the three main clauses in a window function?
7. What is the working of `OVER` clause?
8. What is the working of `PARTITION BY` clause?
9. What is the working of `ORDER BY` clause?
10. What are the different functions that can be applied to windows created by partitioning? 
11. What are the different aggregate functions?
12. What are the different ranking functions?
13. What are the different value functions?
14. What is the working of `ROW_NUMBER()` function?
15. What is the working of `RANK()` function?
16. What is the working of `DENSE_RANK()` function?
17. What is the working of `CUME_DIST()` function?
18. What is the working of `ROW_NUMBER()` function?
19. What is the working of `NTILE()` function?
20. What is the working of `PERCENT_RANK()` function?
21. What is the working of `LAG()` function?
22. What is the working of `LEAD()` function?
23. What is the working of `NTH_VALUE()` function?
24. What are the different advanced windowing functions?
25. What is the working of `ROWS BETWEEN` clause?
26. What is the working of `RANGE BETWEEN` clause?
27. What is `WINDOWS` Alias?

## Solutions for Interview Questions

### Uber Interview Question

> <b> QUESTION </b>: You’re given a dataset of uber rides with the traveling distance (‘distance_to_travel’) and cost (‘monetary_cost’) for each ride. For each date, find the difference between the distance-per-dollar for that date and the average distance-per-dollar for that year-month. Distance-per-dollar is defined as the distance traveled divided by the cost of the ride.

*The output should include the year-month (YYYY-MM) and the absolute average difference in distance-per-dollar (Absolute value to be rounded to the 2nd decimal). 
You should also count both success and failed request_status as the distance and cost values are populated for all ride requests. Also, assume that all dates are unique in the dataset. Order your results by earliest request date first.*

![](https://i.imgur.com/UoNHsF1.png)



```

SELECT b.request_date,
       ROUND(ABS(b.dist_to_cost-b.avg_dist_to_cost)::DECIMAL, 2) as mean_deviation
FROM 
  (SELECT a.request_date,
          a.dist_to_cost,
          AVG(a.dist_to_cost) OVER(PARTITION BY a.request_mnth) AS avg_dist_to_cost
   FROM uber_request_logs) a
 ORDER BY request_date) b
GROUP BY b.request_date,
         b.dist_to_cost,
         b.avg_dist_to_cost
ORDER BY b.request_date

```

### Airbnb Interview Question

> <b> QUESTION </b>: Estimate the growth of Airbnb each year using the number of hosts registered as the growth metric. 

` rate_of_growth = 100 * (hosts_registered_this_year - hosts_registered_last_year) / hosts_registered_last_year `

*Output the year, number of hosts in the current year, number of hosts in the previous year, and the rate of growth. Round the rate of growth to the nearest percent and order the result in the ascending order based on the year.
Assume that the dataset consists only of unique hosts, meaning there are no duplicate hosts listed.*


![](https://i.imgur.com/PSvxbkG.png)


```
SELECT year,
       current_year_host,
       prev_year_host,
       ROUND((current_year_host - prev_year_host)/(CAST(prev_year_host AS numeric)))*100) estimated_growth
FROM
  (SELECT year,
          current_year_host,
          LAG(current_year_host, 1) OVER (ORDER BY year) AS prev_year_host
  FROM 
    (SELECT EXTRACT(year
                    FROM host_since::date) AS year,
            COUNT(id) current_year_host
    FROM airbnb_search_Details
    WHERE host_since IS NOT NULL
    GROUP BY EXTRACT(year
                     FROM host_since::date)
    ORDER BY year) t1) t2     
```

### Twitter Interview Question

> <b> QUESTION </b>: Find the top three unique salaries for each department. Output the department name and the top 3 unique salaries by each department. Order your results alphabetically by department and then by highest salary to lowest.

![](https://i.imgur.com/rrBRmi2.png)

```

SELECT department, 
       salary,
       rank_id
FROM 
  (SELECT department,
          salary,
          RANK() OVER (PARTITION BY a.department
                             ORDER BY a.salary DESC) AS rank_id
  FROM
    (SELECT department, salary
     FROM twitter_employee
     GROUP BY department, salary
     ORDER BY department, salary) a
  ORDER BY department,
           salary DESC)b
WHERE rank_id<4 

```

### Netflix Interview Question

> <b> QUESTION </b>: ABC Corp is a mid-sized insurer in the US and in the recent past their fraudulent claims have increased significantly for their personal auto insurance portfolio. They have developed a ML based predictive model to identify propensity of fraudulent claims. Now, they assign highly experienced claim adjusters for top 5 percentile of claims identified by the model. Your objective is to identify the top 5 percentile of claims from each state. 

*Your output should be policy number, state, claim cost, and fraud score.*

![](https://i.imgur.com/sG64cfJ.png) 


```
SELECT policy_num,
       state,
       claim_cost,
       fraud_score
FROM
  (SELECT *,
          NTILE(100) OVER(PARTITION BY state
                          ORDER BY fraud_score DESC) AS percentile 
  FROM fraud_score) a
WHERE percentile <=5

```